In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


data = pd.read_csv("Over, 600,000 accounts.csv")

In [ ]:
# Data Structuring

data.columns = [i.lower() for i in data.columns]
data.customer_name = data.customer_name.str.lower()
data.address = data.address.str.lower()
data.another_personal_info = data.another_personal_info.str.lower()

In [ ]:
# Consistent Data

data['customer_name'] = data.customer_name.str.replace('org','organization')
data['customer_name'] = data.customer_name.str.replace('#','')
data['customer_name'] = data.customer_name.str.replace(':','')

data['address'] = data.address.str.replace('Postal Code','pc')
data['address'] = data.address.str.replace('post code','pc')

data['another_personal_info'] = data.another_personal_info.str.replace('(','')
data['another_personal_info'] = data.another_personal_info.str.replace(')','')
data['another_personal_info'] = data.another_personal_info.str.replace(' ','')
data['another_personal_info'] = data['another_personal_info'].str[-9:]

In [ ]:
data['address_info'] = data.latitude.astype(str)+ " & " + data.longitude.astype(str)
key = 'primary_key'
col1 = 'contact_info'
col2 = 'address_info'

# LD features and Missing features 
unique = data.dropna(subset=[col1, col2], how='all').drop_duplicates([col1,col2])

In [ ]:
dup_df = pd.DataFrame()
data_temp = data[[key, col1, col2]].copy()
data_temp.fillna('missing',inplace=True)

for i in tqdm(unique[key]):
    col1_value = data_temp.loc[data_temp[key]==i, col1].values[0]
    col2_value = data_temp.loc[data_temp[key]==i, col2].values[0]
    duplicate_ids = data_temp.loc[(data_temp[col2]==col2_value)
                                  &(data_temp[col1]==col1_value)
                                  &(data_temp[key]!=i), 
                                  key].values.tolist()
    if duplicate_ids:
        dup_match = [(i, dup_id) for dup_id in duplicate_ids]
        dup_df = dup_df.append(pd.DataFrame(dup_match))

In [ ]:
def levenshtein_ratio_and_distance(s, t, ratio_calc = True):
    # Initialize matrix of zeros
    try:
        rows = len(s)+1
        cols = len(t)+1
        distance = np.zeros((rows,cols),dtype = int)
        # Populate matrix of zeros with the indeces of each character of both strings
        for i in range(1, rows):
            for k in range(1,cols):
                distance[i][0] = i
                distance[0][k] = k
        # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
        for col in range(1, cols):
            for row in range(1, rows):
                if s[row-1] == t[col-1]:
                    cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
                else:
                    # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                    # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                    if ratio_calc == True:
                        cost = 2
                    else:
                        cost = 1
                distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                     distance[row][col-1] + 1,          # Cost of insertions
                                     distance[row-1][col-1] + cost)     # Cost of substitutions
        if ratio_calc == True:
            # Computation of the Levenshtein Distance Ratio
            Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
            return Ratio
        else:
            # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
            # insertions and/or substitutions
            # This is the minimum number of edits needed to convert string a to string b
            return "The strings are {} edits away".format(distance[row][col])
    except:
        return '''Levenshtine Distance couldn't be calculated'''

In [ ]:
cols = 'customer_name,address,another_personal_info,another_personal_info,another_personal_info,another_personal_info,another_personal_info'.split(",")
dup_df.columns = [key, "Dup_"+key]
dup_match = pd.merge(dup_df, data[cols+[key]], on=key, how='left')
df = pd.merge(dup_match, data[cols+[key]], left_on="Dup_"+ key, right_on=key, how='left')
all_cols = df.columns.tolist()
all_cols.sort()
df = df[all_cols].drop('Dup_partyid',1)
df.head()

In [ ]:
%%time
df['address'] = df.apply(lambda x: levenshtein_ratio_and_distance(x['address_x'], x['address_y']), axis=1)
df['LD_email_address'] = df.apply(lambda x: levenshtein_ratio_and_distance(x['another_personal_info_x'], x['another_personal_info_y']), axis=1)
df['LD_another_personal_info'] = df.apply(lambda x: levenshtein_ratio_and_distance(x['another_personal_info_x'], x['another_personal_info_y']), axis=1)
df['LD_another_personal_info'] = df.apply(lambda x: levenshtein_ratio_and_distance(x['another_personal_info_x'], x['another_personal_info_y']), axis=1)
df['id_match'] = 0
df.loc[df.id_x == df.id_y, 'id_match'] = 1
df.head()

In [ ]:
df = df[['primary_key_x', 'primary_key_y','account_info_x','account_info_y', 'LD_id_info',
         'other_id_x','other_id_y', 'other_id_match','customer_name_x', 'customer_name_y', 
         'LD_customer_name', 'address_x', 'address_y','LD_address', 'another_personal_info_x', 
         'another_personal_info_y','LD_another_personal_info', 'another_personal_info_x',
         'another_personal_info_y', 'address_info_x', 'address_info_y']]

In [ ]:
df['another_personal_info_missing'] = 0 
df.loc[(df.another_personal_info_x.isna())|(df.another_personal_info_y.isna()),'another_personal_info_missing'] = 1
df.loc[(df.another_personal_info_x.isna())&(df.another_personal_info_y.isna()),'another_personal_info_missing'] = 2
df['another_personal_info_missing'] = 0 
df.loc[(df.primarycontactname_x.isna())|(df.primarycontactname_y.isna()),'another_personal_info_missing'] = 1
df.loc[(df.primarycontactname_x.isna())&(df.primarycontactname_y.isna()),'another_personal_info_missing'] = 2
df['another_personal_info_missing'] = 0 
df.loc[(df.formattedphonenumber_x.isna())|(df.formattedphonenumber_y.isna()),'another_personal_info_missing'] = 1
df.loc[(df.formattedphonenumber_x.isna())&(df.formattedphonenumber_y.isna()),'another_personal_info_missing'] = 2

In [ ]:
df.to_excel('duplicate_data.xlsx', index=None)